# Explore Data

In [1]:
import altair as alt
import pandas as pd

from rossmann import load_test, load_train
from rossmann.feature_engineering import compute_competition_open, compute_zscore

In [2]:
pd.set_option("display.float_format", lambda x: "%.3f" % x)
alt.data_transformers.enable("json")  # to allow for large plots

DataTransformerRegistry.enable('json')

In [3]:
df = pd.concat(
    [
        load_train().assign(sample="train"),
        load_test().assign(sample="test"),
    ],
    ignore_index=True,
).sort_values(["store", "date"])

df = df.iloc[:int(.1*len(df))]

In [4]:
# what time horizon are we covering?
df.groupby(["sample"])[["date"]].agg(["count", "min", "max"])

date                      
         count        min        max
sample                              
test      4032 2015-08-01 2015-09-17
train   101797 2013-01-01 2015-07-31

In [5]:
# number of stores
df.groupby(["sample"])[["store"]].agg(lambda x: len(x.unique()))

,store
sample,
test,84
train,112


In [6]:
# are all stores in the test set known?
stores_in_train = df.loc[lambda x: x["sample"].eq("train"), "store"].unique()
stores_in_test = df.loc[lambda x: x["sample"].eq("test"), "store"].unique()
df = df.assign(
    store_in_train=lambda x: x["store"].isin(stores_in_train),
    store_in_test=lambda x: x["store"].isin(stores_in_test),
)
df.groupby(["sample", "store_in_train", "store_in_test"])[["store"]].count()

store
sample store_in_train store_in_test       
test   True           True            4032
train  True           False          25797
                      True           76000

In [7]:
# number of obs per store
df.groupby(["sample", "store"])[["date"]].count().groupby(level=0).agg(
    ["mean", "min", "max"]
)

date          
          mean  min  max
sample                  
test    48.000   48   48
train  908.902  363  942

In [8]:
# number of obs per store
df.groupby(["sample", "store_in_test", "store"])[["date"]].count().groupby(
    level=[0, 1]
).agg(["mean", "min", "max"])


date          
                        mean  min  max
sample store_in_test                  
test   True           48.000   48   48
train  False         921.321  363  942
       True          904.762  758  942

In [9]:
# zero sales when stores are closed
df.groupby(["sample", "open"])[["sales"]].agg(["mean", "min", "max"])

sales                
                 mean   min       max
sample open                          
test   False      NaN   NaN       NaN
       True       NaN   NaN       NaN
train  False    0.000 0.000     0.000
       True  6733.907 0.000 38037.000

In [10]:
# zero sales when stores are open?
df.assign(zero_sales=lambda x: x["sales"].le(1e-7)).groupby(
    ["sample", "zero_sales", "open"]
)[["sales"]].agg(["count"])

sales
                         count
sample zero_sales open        
test   False      False      0
                  True       0
train  False      True   84145
       True       False  17644
                  True       8

In [11]:
# let's define a train set that only contains observations with positive sales
# (consistent with loss function, where these are supposed to be ignored)
df_train = df.loc[lambda x: x["sample"].eq("train") & x["sales"].gt(0)].copy()

In [12]:
# store heterogeneity
alt.Chart(df_train[["store", "sales"]].groupby("store").mean()).mark_bar().encode(
    alt.X("sales:Q", bin=alt.Bin(maxbins=25)),
    y="count()",
)

alt.Chart(...)

In [13]:
df_train[["store", "sales"]].groupby("store").describe(percentiles=[0.01, 0.5, 0.99])

sales                                                           \
        count      mean      std      min       1%       50%       99%   
store                                                                    
1     781.000  4759.096 1012.106 2362.000 3128.000  4647.000  7906.200   
2     784.000  4953.901 1610.149 1919.000 2257.620  4783.000  9560.880   
3     779.000  6942.569 2193.384 2936.000 3399.520  6619.000 12675.760   
4     784.000  9638.402 1936.032 5869.000 6518.150  9430.500 15700.390   
5     779.000  4676.275 1765.746 1423.000 1592.340  4616.000  9072.740   
...       ...       ...      ...      ...      ...       ...       ...   
108   622.000 11035.360 2334.762 5550.000 6865.960 10642.000 17456.540   
109   784.000  6464.324 1477.568 3852.000 4247.960  6137.000 10859.120   
110   784.000  4534.430 1228.500 1944.000 2341.960  4410.000  7781.150   
111   781.000  6771.745 1692.264 2876.000 4017.200  6618.000 11485.400   
112   302.000  6083.765  894.539 4371.000 4724.800  5931.500  8991.030   

                 
            max  
store            
1      9528.000  
2     10682.000  
3     15689.000  
4     17412.000  
5     11692.000  
...         ...  
108   19996.000  
109   14783.000  
110   10064.000  
111   15498.000  
112   12471.000  

[112 rows x 8 columns]

In [14]:
# compute rolling zscore to detect outliers
df_train["zscore"] = compute_zscore(df_train, window=150)
alt.Chart(df_train[["zscore"]]).mark_bar().encode(
    alt.X("zscore:Q", bin=alt.Bin(maxbins=50)),
    y="count()",
)
df_train.loc[lambda x: x["zscore"].abs().gt(5), ["store", "date", "sales", "zscore"]]

,store,date,sales,zscore
9755,11,2013-12-02,16727.000,5.142
9769,11,2013-12-16,20943.000,6.091
10711,12,2013-12-16,16747.000,5.163
15237,17,2013-12-16,17845.000,5.025
16179,18,2013-12-16,14671.000,5.407
20521,23,2013-12-16,13460.000,5.695
21463,24,2013-12-16,21022.000,5.793
21659,24,2014-06-30,22015.000,5.436
22426,25,2014-01-06,24833.000,5.133
23347,26,2013-12-16,16562.000,5.699


In [15]:
# take a look at week-day effects
alt.Chart(df_train[["day_of_week", "sales"]]).mark_boxplot().encode(
    x=alt.X("day_of_week:O"),
    y=alt.Y("sales:Q"),
)

alt.Chart(...)

In [16]:
# take a look at month effects
alt.Chart(df_train[["month", "sales"]]).mark_boxplot().encode(
    x=alt.X("month:O"),
    y=alt.Y("sales:Q"),
)

alt.Chart(...)

In [17]:
# take a look at year effects
alt.Chart(df_train[["year", "sales"]]).mark_boxplot().encode(
    x=alt.X("year:O"),
    y=alt.Y("sales:Q"),
)

alt.Chart(...)

In [18]:
# take a look at promo effects
alt.Chart(df_train[["promo", "sales"]]).mark_boxplot().encode(
    x=alt.X("promo:O"),
    y=alt.Y("sales:Q"),
)


alt.Chart(...)

In [19]:
# take a look at school holiday effects
alt.Chart(df_train[["school_holiday", "sales"]]).mark_boxplot().encode(
    x=alt.X("school_holiday:O"),
    y=alt.Y("sales:Q"),
)


alt.Chart(...)

In [20]:
# take a look at school holiday effects
alt.Chart(df_train[["school_holiday", "sales"]]).mark_boxplot().encode(
    x=alt.X("school_holiday:O"),
    y=alt.Y("sales:Q"),
)


alt.Chart(...)

In [21]:
# competitor open?
df_train["competition_open"] = compute_competition_open(df_train)
alt.Chart(df_train[["competition_open", "sales"]]).mark_boxplot().encode(
    x=alt.X("competition_open:O"),
    y=alt.Y("sales:Q"),
)

alt.Chart(...)